## [DO NOT EDIT] Header

本笔记本实现了用于数字分类的决策树模型，包含超参数调优。
所有代码必须遵循实现指南中定义的项目结构和命名约定。

In [ ]:
# Import required libraries
import sys
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# 魔法命令
%matplotlib inline

# 路径设置 - 导入此模块会自动设置Python路径
import path_setup

# 直接导入项目模块（IDE可以正确追踪）
from load_data import load_digits_dataset
from evaluate import eval_classification, eval_classification_with_roc
from visualize import plot_confusion_matrix, plot_bar

# Constants
RANDOM_STATE = 42
NOTEBOOK_BASENAME = "part4_decision_tree"

# Ensure output directories exist
os.makedirs("../results/figures", exist_ok=True)
os.makedirs("../results/metrics", exist_ok=True)

print("✓ 环境设置完成")


## Dataset Loading

使用提供的API函数加载数字数据集。

In [ ]:
# Load the digits dataset
X_train, X_test, y_train, y_test, target_names = load_digits_dataset()
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Number of classes: {len(target_names)}")

## Model Definition (Default Only)

仅定义一个模型实例，使用库默认参数。

In [ ]:
# Define default model with library defaults
from sklearn.tree import DecisionTreeClassifier
default_model = DecisionTreeClassifier(
    random_state=RANDOM_STATE
)

## Training (Default Model)

对默认参数模型执行训练，记录训练用时。

In [ ]:
# Train the default model
start_time = time.time()
default_model.fit(X_train, y_train)
default_training_time = time.time() - start_time
print(f"Default model training time: {default_training_time:.4f} seconds")

## Evaluation (Default Model)

评估默认模型性能。

In [ ]:
# Evaluate the default model
default_metrics = eval_classification(default_model, X_test, y_test, target_names)
print(f"Default model accuracy: {default_metrics['accuracy']:.4f}")
print(f"Default model macro F1: {default_metrics['macro_f1']:.4f}")
print(f"Default model weighted F1: {default_metrics['weighted_f1']:.4f}")

## Visualization (Default Model)

可视化默认模型结果。

In [ ]:
# Plot confusion matrix for default model
out_png_path = f"../results/figures/{NOTEBOOK_BASENAME}__cm_default.png"
plot_confusion_matrix(default_metrics['confusion_matrix'], target_names, out_png_path)
print(f"Default model confusion matrix saved to {out_png_path}")

## Fill Hyperparams (Tuning Spec)

定义调参网格。

In [ ]:
# Define hyperparameter grid for Decision Tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define base model with stability parameters
base_model = DecisionTreeClassifier(
    random_state=RANDOM_STATE
)

# Define parameter grid (strictly follow Canvas Chapter 4)
param_grid = {
    'max_depth': [3, 5, 7, 9, None],
    'min_samples_split': [2, 5, 10]
}

# Create GridSearchCV object
gs = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

# Execute grid search
gs.fit(X_train, y_train)

# Print best parameters and CV score
print("Best params:", gs.best_params_)
print("Best CV score (accuracy):", gs.best_score_)

# Get best model and evaluate on test set
best_model = gs.best_estimator_
test_acc = best_model.score(X_test, y_test)
print("Test accuracy with best params:", test_acc)

# Save variables for later use
model = best_model
best_params = gs.best_params_
best_training_time = gs.cv_results_['mean_fit_time'][gs.best_index_]

## Training (Tuned Model)

使用最佳参数实例化第二个模型对象并训练，记录用时。

In [ ]:
# Train the best model (already done during grid search)
print(f"Best model trained with hyperparameters: {best_params}")
print(f"Training time: {best_training_time:.4f} seconds")

## Evaluation (Tuned Model)

评估调参后模型性能。

In [ ]:
# Evaluate the best model
final_metrics = eval_classification(best_model, X_test, y_test, target_names)

print(f"Accuracy: {final_metrics['accuracy']:.4f}")
print(f"Macro F1: {final_metrics['macro_f1']:.4f}")
print(f"Weighted F1: {final_metrics['weighted_f1']:.4f}")
print("\nClassification Report:")
print(final_metrics['classification_report'])

## Visualization (Tuned Model + Param Curves)

可视化调参后模型结果和性能曲线。

In [ ]:
# ① Plot confusion matrix for tuned model
cm = final_metrics['confusion_matrix']
out_png_path = f"../results/figures/{NOTEBOOK_BASENAME}__cm_tuned.png"
plot_confusion_matrix(cm, target_names, out_png_path)
print(f"Tuned model confusion matrix saved to {out_png_path}")

# ② Plot parameter performance curves
cv_results = gs.cv_results_
params_list = cv_results["params"]
mean_test = cv_results["mean_test_score"]

# Plot each parameter
for param_key in param_grid.keys():
    if param_key == 'max_depth':
        # Use best min_samples_split
        best_mss = best_params['min_samples_split']
        depth_vals = []
        scores = []
        for i, params in enumerate(params_list):
            if params['min_samples_split'] == best_mss:
                depth_vals.append(params['max_depth'])
                scores.append(mean_test[i])
        
        # Sort, handling None
        def sort_key(x):
            return (999 if x[0] is None else x[0])
        sorted_pairs = sorted(zip(depth_vals, scores), key=sort_key)
        depth_vals_sorted = [p[0] for p in sorted_pairs]
        scores_sorted = [p[1] for p in sorted_pairs]
        
        # Convert None to string for plotting
        depth_labels = [str(d) if d is not None else 'None' for d in depth_vals_sorted]
        x_positions = range(len(depth_labels))
        
        # Plot
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(x_positions, scores_sorted, marker='o', linewidth=2, markersize=8)
        ax.set_xticks(x_positions)
        ax.set_xticklabels(depth_labels)
        ax.set_xlabel('max_depth', fontsize=12)
        ax.set_ylabel('Cross-Validation Accuracy', fontsize=12)
        ax.set_title(f'Decision Tree: Accuracy vs max_depth (min_samples_split={best_mss})', fontsize=14)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        out_png_path = f"../results/figures/{NOTEBOOK_BASENAME}__acc_vs_max_depth.png"
        plt.savefig(out_png_path, dpi=100, bbox_inches='tight')
        plt.show()
        print(f"Accuracy vs max_depth plot saved to {out_png_path}")
    
    elif param_key == 'min_samples_split':
        # Use best max_depth
        best_depth = best_params['max_depth']
        mss_vals = []
        scores = []
        for i, params in enumerate(params_list):
            if params['max_depth'] == best_depth:
                mss_vals.append(params['min_samples_split'])
                scores.append(mean_test[i])
        
        # Sort
        sorted_pairs = sorted(zip(mss_vals, scores))
        mss_vals_sorted = [p[0] for p in sorted_pairs]
        scores_sorted = [p[1] for p in sorted_pairs]
        
        # Plot
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(mss_vals_sorted, scores_sorted, marker='o', linewidth=2, markersize=8)
        ax.set_xlabel('min_samples_split', fontsize=12)
        ax.set_ylabel('Cross-Validation Accuracy', fontsize=12)
        ax.set_title(f'Decision Tree: Accuracy vs min_samples_split (max_depth={best_depth})', fontsize=14)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        out_png_path = f"../results/figures/{NOTEBOOK_BASENAME}__acc_vs_min_samples_split.png"
        plt.savefig(out_png_path, dpi=100, bbox_inches='tight')
        plt.show()
        print(f"Accuracy vs min_samples_split plot saved to {out_png_path}")

## Persist Metrics

将指标保存到指定的JSON文件。

In [ ]:
# Create metrics dictionary
metrics_dict = {
    "model_name": "Decision Tree",
    "best_hyperparams": best_params,
    "accuracy": final_metrics['accuracy'],
    "macro_f1": final_metrics['macro_f1'],
    "weighted_f1": final_metrics['weighted_f1'],
    "train_time_sec": best_training_time,
    "notes": f"Decision Tree with max_depth={best_params['max_depth']}, min_samples_split={best_params['min_samples_split']} achieved {final_metrics['accuracy']:.4f} accuracy"
}

# Save metrics to JSON file
metrics_path = f"../results/metrics/{NOTEBOOK_BASENAME}__metrics.json"
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics_dict, f, indent=2)

print(f"Metrics saved to {metrics_path}")
print(f"Final metrics: {metrics_dict}")

使用 max_depth=9、min_samples_split=2 的决策树达到了最佳性能，准确率为 0.8528。
本模型对 max_depth 参数敏感，适中的树深度在该数据集上效果更好。


## Conclusion (Template)